In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("lmsys/lmsys-arena-human-preference-55k")
df = ds['train'].to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/57477 [00:00<?, ? examples/s]

# Inspection

In [ ]:
df.columns

Index(['id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b',
       'winner_model_a', 'winner_model_b', 'winner_tie'],
      dtype='object')

In [ ]:
df.head(100)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
95,6831350,llama-2-7b-chat,zephyr-7b-beta,"[""What inspired Leonardo da Vinci to create hi...","[""Leonardo da Vinci's \""Mona Lisa\"" is one of ...","[""The exact inspiration behind Leonardo da Vin...",1,0,0
96,6853605,pplx-70b-online,gpt-4-0314,"[""What's funnier than 420?""]","[""The question \""What's funnier than 420?\"" is...","[""Funnier than 420 could be the number \""69\"" ...",0,1,0
97,7021095,gpt-4-1106-preview,vicuna-13b,"[""how can I get minecraft java to run better? ...","[""Improving the performance of Minecraft Java ...","[""Here are some tips to improve the performanc...",0,0,1
98,7047857,gpt-4-1106-preview,claude-2.1,"[""If you were a type of cheese, which one woul...","[""If I were to liken myself to a type of chees...","[""I'm an AI assistant without personal prefere...",1,0,0


### Tied comparisons

In [ ]:
tied = len(df[df['winner_tie']==1])
total = len(df)

print(f'Total comparisons = {total}')
print(f'Number of tied comparisons = {tied} ({round(tied/total*100, 2)}%)')

Total comparisons = 57477
Number of tied comparisons = 17761 (30.9%)


### Number of prompts in each conversation. Some have over 20 prompts!

In [ ]:
prompts = [eval(p) for p in df['prompt']]
prompt_lengths = [len(p) for p in prompts]
for l in set(prompt_lengths):
  print(f'Number of conversations with {l} prompts: {prompt_lengths.count(l)}')

Number of conversations with 1 prompts: 49938
Number of conversations with 2 prompts: 4673
Number of conversations with 3 prompts: 1485
Number of conversations with 4 prompts: 607
Number of conversations with 5 prompts: 311
Number of conversations with 6 prompts: 167
Number of conversations with 7 prompts: 103
Number of conversations with 8 prompts: 60
Number of conversations with 9 prompts: 44
Number of conversations with 10 prompts: 29
Number of conversations with 11 prompts: 16
Number of conversations with 12 prompts: 14
Number of conversations with 13 prompts: 5
Number of conversations with 14 prompts: 3
Number of conversations with 15 prompts: 4
Number of conversations with 16 prompts: 2
Number of conversations with 17 prompts: 3
Number of conversations with 18 prompts: 1
Number of conversations with 19 prompts: 4
Number of conversations with 20 prompts: 2
Number of conversations with 21 prompts: 1
Number of conversations with 22 prompts: 1
Number of conversations with 29 prompts:

### Number of prompts match the number of responses, which makes sense

In [ ]:
response_a = [eval(r.replace('null', 'None')) for r in df['response_a']]
response_b = [eval(r.replace('null', 'None')) for r in df['response_b']]

response_a_lengths = [len(r) for r in response_a]
response_b_lengths = [len(r) for r in response_b]

for l in set(response_a_lengths):
  print(f'Number of conversations with {l} responses in response_a: {response_a_lengths.count(l)}')

Number of conversations with 1 responses in response_a: 49938
Number of conversations with 2 responses in response_a: 4673
Number of conversations with 3 responses in response_a: 1485
Number of conversations with 4 responses in response_a: 607
Number of conversations with 5 responses in response_a: 311
Number of conversations with 6 responses in response_a: 167
Number of conversations with 7 responses in response_a: 103
Number of conversations with 8 responses in response_a: 60
Number of conversations with 9 responses in response_a: 44
Number of conversations with 10 responses in response_a: 29
Number of conversations with 11 responses in response_a: 16
Number of conversations with 12 responses in response_a: 14
Number of conversations with 13 responses in response_a: 5
Number of conversations with 14 responses in response_a: 3
Number of conversations with 15 responses in response_a: 4
Number of conversations with 16 responses in response_a: 2
Number of conversations with 17 responses 

In [ ]:
response_a_lengths == response_b_lengths

True

In [ ]:
df['processed_prompts'] = prompts
df['processed_response_a'] = response_a
df['processed_response_b'] = response_b

# Embedding

Approach 1: Just take the pairs with 1 prompt, 1 response \\
Approach 2: Make all prompt-response pairs independent of the conversation they're part of (which doesn't make a lot of sense as the winning symbol is given to the entire conversation, not just a prompt-response pair) \\
Approach 3: Create pair-wise sequence embeddings that will have the entire conversation as 1 embedding.

### Text Cleaning

In [ ]:
!pip install nltk

In [ ]:
import nltk
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# need to clean more - like remove digits, any words less than 2 characters, etc
def clean_text(text):
    if text is None:
        return ''
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    tokens = word_tokenize(text.lower())
    cleaned_tokens = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in string.punctuation and token not in stop_words
    ]

    return ' '.join(cleaned_tokens)


In [ ]:
def clean_conversation_column(column):
    return column.apply(lambda x: [clean_text(text) for text in x])

df['cleaned_prompts'] = clean_conversation_column(df['processed_prompts'])
df['cleaned_response_a'] = clean_conversation_column(df['processed_response_a'])
df['cleaned_response_b'] = clean_conversation_column(df['processed_response_b'])


## Approach 1

In [ ]:
df['prompt_length'] = prompt_lengths
filtered_df = df[df['prompt_length']==1]

len(filtered_df)

49938

In [ ]:
!pip install -U sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import numpy as np

def create_single_turn_embeddings(data, model):
    embeddings_a = []
    embeddings_b = []

    for i, row in data.iterrows():
      print(i)
      prompt = row['cleaned_prompts'][0]
      response_a = row['cleaned_response_a'][0]
      response_b = row['cleaned_response_b'][0]

      embedding_a = model.encode(prompt + " " + response_a)
      embedding_b = model.encode(prompt + " " + response_b)

      embeddings_a.append(embedding_a)
      embeddings_b.append(embedding_b)

    return np.array(embeddings_a), np.array(embeddings_b)

embeddings_a, embeddings_b = create_single_turn_embeddings(filtered_df, model)
np.save('single_embeddings_a.npy', embeddings_a)
np.save('single_embeddings_b.npy', embeddings_b)

Streaming output truncated to the last 5000 lines.
51705
51706
51707
51708
51709
51710
51711
51713
51716
51717
51718
51720
51721
51722
51725
51726
51727
51728
51729
51730
51731
51733
51735
51736
51738
51739
51740
51741
51743
51744
51745
51746
51747
51748
51749
51750
51751
51753
51756
51757
51759
51760
51761
51762
51763
51764
51766
51767
51768
51769
51770
51771
51773
51774
51775
51776
51777
51778
51779
51780
51783
51784
51785
51787
51788
51789
51790
51791
51792
51793
51794
51795
51796
51797
51798
51799
51800
51801
51802
51803
51804
51806
51808
51809
51811
51812
51813
51814
51815
51817
51818
51819
51820
51821
51822
51824
51825
51826
51827
51828
51829
51830
51831
51833
51834
51835
51837
51838
51839
51840
51842
51844
51845
51847
51849
51850
51851
51852
51853
51854
51855
51856
51857
51859
51860
51861
51862
51863
51864
51866
51867
51868
51869
51870
51871
51872
51873
51874
51875
51877
51878
51879
51880
51881
51882
51883
51884
51886
51888
51889
51890
51891
51892
51893
51894
51895
51896
51897
5

## Aprroach 3

In [ ]:
def create_multi_turn_embeddings(data, model):
  embeddings_a = []
  embeddings_b = []

  for i, row in data.iterrows():
    print(i)
    context = ""
    context_embeddings_a = []
    context_embeddings_b = []

    for i in range(len(row['cleaned_prompts'])):
      if i == 0:
          context += row['cleaned_prompts'][i]
      else:
          context += " " + row['cleaned_prompts'][i]

      response_a = row['cleaned_response_a'][i]
      response_b = row['cleaned_response_b'][i]

      embedding_a = model.encode(context + " " + response_a)
      embedding_b = model.encode(context + " " + response_b)

      context_embeddings_a.append(embedding_a)
      context_embeddings_b.append(embedding_b)

    embeddings_a.append(np.mean(context_embeddings_a, axis=0))
    embeddings_b.append(np.mean(context_embeddings_b, axis=0))

  return np.array(embeddings_a), np.array(embeddings_b)

embeddings_a, embeddings_b = create_multi_turn_embeddings(df, model)
np.save('multi_embeddings_a.npy', embeddings_a)
np.save('multi_embeddings_b.npy', embeddings_b)